# Project Title (w/ header below)

## Sources
This project makes use of information from the following sources:
  
* Dr. Sprint's [CPSC 222 Course Materials](https://github.com/GonzagaCPSC222)

* Pandas [Documentation Pages](https://pandas.pydata.org/docs/)

* [Python Basics](https://pythonbasics.org/)

Links to specific pages are included throughout as they are relavent.

## Introduction

## Data Preperation
The Fitbit website allows access to personal data in a very friendly format. I have access to both my sleep and activity data gathered by my device from September 17, 2020 to the present. Looking at the data using Microsoft Excel, I can see that every entry is a numerical value, except for the date and time stamps. A few of the columns will also have to be converted from strings to integers due to the prescence of commas. The activity CSV has one date column for which every other attribute has a corresponding value. The sleep CSV meanwhile, has two timestamps columns: one for the sleep start time and one for the sleep stop time. That will be my main task in preparing the data. 
### Cleaning (Spreadsheet)
Looking at both CSV files in Microsoft Excel, I can see that both top rows contain one cell that has the label for the data table.  
<img src="https://github.com/lmartin5/CPSC_222_Quantified_Self_Project/blob/master/figures/data_with_label_row.PNG?raw=true"  width="700"/>  
Since this is a one-step process, for both CSV files I manually deleted the top row, so that now the top row only contains the column labels.  
<img src="https://github.com/lmartin5/CPSC_222_Quantified_Self_Project/blob/master/figures/data_without_label_row.PNG?raw=true"  width="700"/>

### Loading the Data
`Pandas` will be the main module used for working directly with the data, and the `read_csv()` function can be used to load the data directly. The `index_col` key word in the `read_csv()` function can be set to 0 when loading in the activity data because it already has the date in a usable format. More work will have to be done using the time stamps in the sleep data to get a identical index column. The `set_option()` function is used to force the program to display all of the columns of the DataFrame as discussed [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.set_option.html).

In [1]:
import pandas as pd

sleep_df = pd.read_csv("files/fitbit_sleep_data.csv")
activity_df = pd.read_csv("files/fitbit_activity_data.csv", index_col=0)
pd.set_option("display.max_columns", None)

There are three columns in the sleep DataFrame that only contain NaN values. These are attributes that are tracked by devices more advanced than the one used in this project, so they can be droped from the DataFrame. I will also change the column labels so that the DataFrames can print out in a more friendly format in the report. 

In [2]:
sleep_df.drop(["Minutes REM Sleep", "Minutes Light Sleep", "Minutes Deep Sleep"], axis=1, inplace=True)
sleep_df.columns = ["Start", "End", "Asleep", "Awake", "Times Awake", "Time in Bed"]
activity_df.columns = ["Cals", "Steps", "Distance", "Floors", "MS", "MLA", "MFA", "MVA", "Act. Cals"]

print("Sleep Data")
print(sleep_df.head())
print("Activity Data")
print(activity_df.head())

Sleep Data
                Start                 End  Asleep  Awake  Times Awake  \
0   2020-11-19 1:22AM   2020-11-19 8:47AM     422     14            2   
1  2020-11-17 11:56PM   2020-11-18 7:02AM     398     28            1   
2  2020-11-17 12:22AM   2020-11-17 7:08AM     378     24            1   
3  2020-11-16 12:27AM   2020-11-16 6:33AM     349     17            1   
4   2020-11-15 1:59AM  2020-11-15 11:21AM     525     37            1   

   Time in Bed  
0          444  
1          426  
2          406  
3          366  
4          562  
Activity Data
            Cals   Steps  Distance  Floors     MS  MLA  MFA  MVA Act. Cals
Date                                                                      
9/17/2020  1,904     353      0.17       0  1,407   33    0    0       128
9/18/2020  3,125  11,504      5.55       5    647  295   39   31     1,666
9/19/2020  2,338   3,463      1.67       2    720  147    0    0       577
9/20/2020  2,777   8,782      4.23       5    528  210   24

### Type Conversion
Right now, there are a few columns in the activity DataFrame that hold strings instead of integers due to commas. I wrote a function to deal with these commas, and it can be used on the columns that are currently strings.

In [3]:
def clean_column(column):
    '''
    Cleans a column that contains strings of integer values with commas
    Parameter column: a Pandas Series from a DataFrame
    Returns: a Series with the same data but of type int
    '''
    col_copy = column.copy()
    for i in col_copy.index:
        str_copy = col_copy[i]
        str_copy = str_copy.replace(",", "")
        col_copy[i] = str_copy
    col_copy = col_copy.astype(int)
    return col_copy

activity_df["Cals"] = clean_column(activity_df["Cals"])
activity_df["Steps"] = clean_column(activity_df["Steps"])
activity_df["MS"] = clean_column(activity_df["MS"])
activity_df["Act. Cals"] = clean_column(activity_df["Act. Cals"])

I can confirm now that every column consists of numerical data using the DataFrame `dtypes` attribute.

In [4]:
print(sleep_df.dtypes)
print(activity_df.dtypes)

Start          object
End            object
Asleep          int64
Awake           int64
Times Awake     int64
Time in Bed     int64
dtype: object
Cals           int32
Steps          int32
Distance     float64
Floors         int64
MS             int32
MLA            int64
MFA            int64
MVA            int64
Act. Cals      int32
dtype: object


### Manipulating Time Stamps
I want the instances of the activity DataFrame and the instances of the sleep DataFrame to line up so that I can compare them. For this project, I want to compare my sleep data for each day to my activity data for the day preceding the sleep. First, I need to get just the desired date from the time stamps, and then get rid of the two time stamp columns. Because I sometimes went to sleep before midnight and sometimes after midnight, I need to only use the "End Time" column. The dash characters need to be [replaced](https://pythonbasics.org/replace/) with forward slash charcters just like in the activity DataFrame. After that, I have to [sort](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html) the order of the sleep DataFrame so that it is in ascending chronological order similar to the activity DataFrame. I am going to have to also shift the dates over one day, so that the first data point in the activity data lines up with the first data point in the sleep data; the first index for each should be September 17, 2020.

In [5]:
index = sleep_df["End"].copy() 
sleep_df.drop(["Start", "End"], axis=1, inplace=True)
index_len = len(index)
for i in range(index_len):
    str_copy = index[i]
    str_copy = str_copy[0:10] # retrieves the first 10 characters, which is the date format
    str_copy = str_copy.replace("-", "/")
    index[i] = str_copy
index.name = "Date"
sleep_df.index = index
sleep_df.sort_index(inplace=True) # reverses the order of the DataFrame
# Now that both DataFrames are in the same order, I can make them have identical indices
sleep_df.index = activity_df.index
print(sleep_df.head(30))

            Asleep  Awake  Times Awake  Time in Bed
Date                                               
9/17/2020      404     24            1          428
9/18/2020      526     47            3          573
9/19/2020      585     10            0          596
9/20/2020      430     29            1          462
9/21/2020      388      7            0          396
9/22/2020      486     22            1          508
9/23/2020      484     11            0          495
9/24/2020      415     20            2          435
9/25/2020      483     40            2          526
9/26/2020      493     28            0          521
9/27/2020      349     40            3          389
9/28/2020      378     11            1          389
9/29/2020      427     18            1          445
9/30/2020      406     18            1          430
10/1/2020      384      3            0          387
10/2/2020      608     23            0          631
10/3/2020      426     10            0          436
10/4/2020   

To confirm that both DataFrames are now indexed the way that I want them, I will look at a random date: October 1st. I want the 10/1/2020 activity data to line up with the amount of sleep when I woke up on October 2nd. Looking at the above sleep DataFrame, I can see that at the index 10/01/2020 there are values of 384, 3, 0, and 387. I have confirmed that these are the same numbers in the sleep CSV file that correspond with the instance where I woke up on October 2nd. I now have the data in a format where I can directly compare them.

### Data Merging
The last thing to do in data preperation is to [join](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) the data into one DataFrame. I am also exporting the data to a CSV file so it can be saved and referenced during the project.

In [6]:
sleep_activity_df = activity_df.join(sleep_df)
print(sleep_activity_df.head())
sleep_activity_df.to_csv("files/fitbit_data.csv")

           Cals  Steps  Distance  Floors    MS  MLA  MFA  MVA  Act. Cals  \
Date                                                                       
9/17/2020  1904    353      0.17       0  1407   33    0    0        128   
9/18/2020  3125  11504      5.55       5   647  295   39   31       1666   
9/19/2020  2338   3463      1.67       2   720  147    0    0        577   
9/20/2020  2777   8782      4.23       5   528  210   24   32       1230   
9/21/2020  3663  17612      8.49      26   596  267  118   47       2284   

           Asleep  Awake  Times Awake  Time in Bed  
Date                                                
9/17/2020     404     24            1          428  
9/18/2020     526     47            3          573  
9/19/2020     585     10            0          596  
9/20/2020     430     29            1          462  
9/21/2020     388      7            0          396  


## Instance Labeling